> Código pendiente para revisión

# Chapter 18: Indexando y Seleccionando datos
### Examples
Select column by label

In [ ]:
import pandas as pd
import numpy as np

# Creando una muestra de DF
df = pd.DataFrame(np.random.randn(5, 3), columns=list('ABC'))
# Show DF
df


In [ ]:

# Seleccionando una columna usando la etiqueta, 'A'
df['A']


In [ ]:

# Seleccionando multiples columnas usando una lista de etiquetas, ['A', 'C']
df[['A', 'C']]


Additional details at: http://pandas.pydata.org/pandas-docs/version/0.18.0/indexing.html#selection-by-label

Select by position

The iloc (short for integer location) method allows to select the rows of a dataframe based on their position index. This way one can slice dataframes just like one does with Python's list slicing.

[Detalles adicionales](http://pandas.pydata.org/pandas-docs/version/0.18.0/indexing.html#selection-by-label)

#### Seleccionar por posición

El método iloc (abreviatura de ubicación entera) permite seleccionar las filas de un dataframe basado en su índice de posición. De esta manera, uno puede dividir dataframes de la misma forma en que se hace con el corte de listas de Python.

In [ ]:
df = pd.DataFrame([[11, 22], [33, 44], [55, 66]], index=list("abc"))
df

df.iloc[0] # the 0th index (row)

In [ ]:
df.iloc[1] # the 1st index (row)

In [ ]:
df.iloc[:2] # the first 2 rows

In [ ]:
df[::-1] # reverse order of rows

Row location can be combined with column location

In [ ]:
df.iloc[:, 1] # the 1st column

Tambien vea: [Selección por posición](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-integer)

Slicing con etiquetas

Cuando usas etiquetas, ambos, el inicio y la parada son incluidos en los resultados.

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(5)
df = pd.DataFrame(np.random.randint(100, size=(5, 5)), columns = list("ABCDE"),
    index = ["R" + str(i) for i in range(5)])

Filas R0 a R2:

In [ ]:
df.loc['R0':'R2']

Note como `loc` difiere de `iloc` debido a que `iloc` excluye el indice final

In [ ]:
df.loc['R0':'R2'] # rows labelled R0, R1, R2

# df.iloc[0:2] # rows indexed by 0, 1

Columnas C a D:

In [ ]:
df.loc[:,'C':'E']

#### Posiciones mezcladas y etiquetas basadas en selección.

DataFrame:

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(5)
df = pd.DataFrame(np.random.randint(100, size=(5, 5)), columns = list("ABCDE"),
    index = ["R" + str(i) for i in range(5)])

Seleccion de filas por posicion, y columnas por etiquetas:

In [ ]:
df.ix[1:3, 'C':'E']

Si el índice es un entero, .ix usará etiquetas en lugar de posiciones:

In [ ]:
df.index = np.arange(5, 10)
df


#same call returns an empty DataFrame because now the index is integer
df.ix[1:3, 'C':'E']

#### Indexado booleano.

Uno puede seleccionar filas y columnas de un datafrae usando arrays booleanos.

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(5)
df = pd.DataFrame(np.random.randint(100, size=(5, 5)), columns = list("ABCDE"),
    index = ["R" + str(i) for i in range(5)])
print (df)

In [ ]:
mask = df['A'] > 10
print (mask)

In [ ]:
print(df[mask])

In [ ]:
print (df.ix[mask, 'C'])

In [ ]:
print(df.ix[mask, ['C', 'D']])

Mas en la [documentación de pandas](http://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing)

#### Filtrando columnas (seleccionando lo interesante y descartando lo innecesario, usando RegEx, etc)

---

Generando un DF de muestra

In [ ]:
df = pd.DataFrame(np.random.randint(0, 10, size=(5, 6)),
        columns=['a10','a20','a25','b','c','d'])
df

Mostrando columnas que contengan la letra "a"

In [ ]:
df.filter(like='a')

Mostrando columnas usando el filtro RegEx `(a|c|d)` - `b` o `c` o `d`

In [ ]:
df.filter(regex='(b|c|d)')

Mostrando todas las columnas excepto las que comienzan con `a`

En otras palabras remueve o descarta todas las columnas satisfaciendo lo dado por el RegEx 

In [ ]:
df.ix[:, ~df.columns.str.contains('^a')]

#### Filtrando / seleccionando filas usando el metodo `.query()`

Generando un DF aleatorio

In [ ]:
import pandas as pd

df = pd.DataFrame(np.random.randint(0,10,size=(10, 3)), columns=list('ABC'))
print(df)

Seleccionando filas donde los valores en columna `A > 2 y B < 5`

In [ ]:
df.query('A > 2 and B < 5')

Usando el método `.query()` con variables para filtrar

In [ ]:
B_filter = [1,7]
df.query('B == @B_filter')

df.query('@B_filter in B')

#### Corte Dependiente del Camino

Puede ser necesario recorrer los elementos de una serie o las filas de un dataframe de una manera en que el siguiente elemento o fila dependa del elemento o fila previamente seleccionado. Esto se llama dependencia de camino.

Considera la siguiente serie de tiempo s con frecuencia irregular.

In [ ]:
#starting python community conventions
import numpy as np
import pandas as pd

# n is number of observations
n = 5000
day = pd.to_datetime(['2013-02-06'])

# irregular seconds spanning 28800 seconds (8 hours)
seconds = np.random.rand(n) * 28800 * pd.Timedelta(1, 's')

# start at 8 am
start = pd.offsets.Hour(8)

# irregular timeseries
tidx = day + start + seconds
tidx = tidx.sort_values()
s = pd.Series(np.random.randn(n), tidx, name='A').cumsum()
s.plot();

Supongamos una condición dependiente del camino. Comenzando con el primer elemento de la serie, quiero seleccionar cada elemento subsiguiente de manera que la diferencia absoluta entre ese elemento y el elemento actual sea mayor o igual a x.

Resolveremos este problema utilizando generadores de Python.

#### Función generadora

In [ ]:
def mover(s, move_size=10):
    """Given a reference, find next value with
    an absolute difference >= move_size"""
    ref = None
    for i, v in s.iteritems():
        if ref is None or (abs(ref - v) >= move_size):
            yield i, v
            ref = v

Entonces nosotros podemos definir una nueva serie como `moves`

In [ ]:
moves = pd.Series({i:v for i, v in mover(s, move_size=10)},
        name='_{}_'.format(s.name))

Graficando ambos

In [ ]:
moves.plot(legend=True)
s.plot(legend=True)

El equivalente para dataframes sería:

In [ ]:
def mover_df(df, col, move_size=2):
    ref = None
    for i, row in df.iterrows():
        if ref is None or (abs(ref - row.loc[col]) >= move_size):
            yield row
            ref = row.loc[col]

df = s.to_frame()
moves_df = pd.concat(mover_df(df, 'A', 10), axis=1).T

moves_df.A.plot(label='_A_', legend=True)
df.A.plot(legend=True)

Obtener las primeras/últimas n filas de un dataframe

- Para ver los primeros o últimos registros de un dataframe, puedes utilizar los métodos `head y tail`.
- Para devolver las primeras n filas, usa `DataFrame.head([n])`.

In [ ]:
df.head(n)

Para retornar las ultimas n filas use `DataFrame.tail([n])`

In [ ]:
df.tail(n)

Sin el argumento n, estas funciones retornan 5 filas.

> Ten en cuenta que la notación de corte para head/tail sería:

In [ ]:
df[:10] # same as df.head(10)
df[-10:] # same as df.tail(10)

#### Seleccionar filas distintas en todo el dataframe.

Let

In [ ]:
df['col_1'].unique()

Pero `Series.unique()` funciona solo para una sola columna.

Para simular el select unique col_1, col_2 de SQL, puedes usar `DataFrame.drop_duplicates()`:

In [ ]:
df.drop_duplicates()

Esto obtendrá todas las filas unicas en el dataframe.

In [ ]:
df = pd.DataFrame({'col_1':['A','B','A','B','C'], 'col_2':[3,4,3,5,6],
                    'col_3':[0,0.1,0.2,0.3,0.4]})
df

In [ ]:
df.drop_duplicates()

Para especificar la columna a considerar cuando seleccionamos registros unicos, pasaremos estos argumentos.

In [ ]:
df = pd.DataFrame({'col_1':['A','B','A','B','C'], 'col_2':[3,4,3,5,6],
                    'col_3':[0,0.1,0.2,0.3,0.4]})
df.drop_duplicates(['col_1','col_2'])

#
#
# skip last column
df.drop_duplicates(['col_1','col_2'])[['col_1','col_2']]

Fuente: [¿Como seleccionar unicos a traves de multiples ccolumnas en dataframes de pandas?](http://stackoverflow.com/questions/30530663/how-to-select-distinct-across-multiple-data-frame-columns-in-pandas)

#### Filtrar las filas con datos faltantes (NaN, None, NaT)

Si tienes un dataframe con datos faltantes `(NaN, pd.NaT, None)`, puedes filtrar las filas incompletas.

In [ ]:
df = pd.DataFrame([[0,1,2,3],
                    [None,5,None,pd.NaT],
                    [8,None,10,None],
                    [11,12,13,pd.NaT]],columns=list('ABCD'))
df

[DataFrame.dropna](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) Descarta todas las filas que almenos tenga un campo con datos perdidos.

In [ ]:
df.dropna()

Para justo descartar las filas que estan perdidos en una columna especifica use `subset`

In [ ]:
df.dropna(subset=['C'])

Utiliza la opción `inplace = True` para reemplazo en su lugar con el marco filtrado.

Lee: [Indexing and selecting data online](https://riptutorial.com/pandas/topic/1751/indexing-and-selecting-data)